# Similarity Evaluation Analysis (SEA) Dataset C

In [1]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
print('Libraries imported!!')

Libraries imported!!


In [2]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/PRIVACY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for univariate resemblance analisys
from similarity_evaluation import scale_data
from similarity_evaluation import pairwise_euclidean_distance
from similarity_evaluation import hausdorff_distance
from similarity_evaluation import rts_similarity

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read real and synthetic datasets
In this part real and synthetic datasets are read.

In [3]:
#Define global variables
DATA_TYPES = ['Real','GM','SDV','CTGAN','WGANGP']
SYNTHESIZERS = ['GM','SDV','CTGAN','WGANGP']
FILEPATHS = {'Real' : HOME_PATH + 'REAL DATASETS/TRAIN DATASETS/C_Obesity_Data_Real_Train.csv',
            'GM' : HOME_PATH + 'SYNTHETIC DATASETS/GM/C_Obesity_Data_Synthetic_GM.csv',
            'SDV' : HOME_PATH + 'SYNTHETIC DATASETS/SDV/C_Obesity_Data_Synthetic_SDV.csv',
            'CTGAN' : HOME_PATH + 'SYNTHETIC DATASETS/CTGAN/C_Obesity_Data_Synthetic_CTGAN.csv',
            'WGANGP' : HOME_PATH + 'SYNTHETIC DATASETS/WGANGP/C_Obesity_Data_Synthetic_WGANGP.csv'}
categorical_columns = ['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','SCC','CALC','MTRANS','Obesity_level']
data = dict()

In [4]:
#iterate over all datasets filepaths and read each dataset
for name, path in FILEPATHS.items() :
    data[name] = pd.read_csv(path)
    for col in categorical_columns :
        data[name][col] = data[name][col].astype('category').cat.codes
data

{'Real':       Gender  Age  Height  Weight  family_history_with_overweight  FAVC  \
 0          0   21    1.63   60.00                               1     1   
 1          0   21    1.75  133.62                               1     1   
 2          0   23    1.66   82.60                               1     1   
 3          0   22    1.59   44.24                               0     0   
 4          1   26    1.81  106.04                               1     1   
 ...      ...  ...     ...     ...                             ...   ...   
 1683       1   32    1.75  120.10                               1     1   
 1684       1   23    1.72   81.67                               1     1   
 1685       0   23    1.65   80.00                               1     1   
 1686       0   23    1.63   84.50                               1     1   
 1687       1   20    1.82   85.00                               1     1   
 
           FCVC       NCP  CAEC  SMOKE      CH2O  SCC       FAF       TUE  \
 

## 2. Normalize data

In [5]:
#Scale the data
num_cols = (data['Real'].select_dtypes(include=['int64','float64'])).columns
scaled_data = dict()
for name in DATA_TYPES :
    scaled_data[name] = scale_data(data[name][num_cols])
scaled_data

{'Real':            Age    Height    Weight      FCVC       NCP      CH2O       FAF  \
 0     0.148936  0.339623  0.156716  1.000000  0.666667  0.500000  0.666667   
 1     0.148936  0.566038  0.706119  1.000000  0.666667  0.943830  0.493640   
 2     0.191489  0.396226  0.325373  0.101877  0.118451  0.882796  0.042781   
 3     0.170213  0.264151  0.039104  1.000000  0.232027  0.775154  0.366287   
 4     0.255319  0.679245  0.500299  1.000000  0.666667  0.929086  0.604439   
 ...        ...       ...       ...       ...       ...       ...       ...   
 1683  0.382979  0.566038  0.605224  0.983650  0.666667  0.765017  0.318439   
 1684  0.191489  0.509434  0.318433  0.500000  0.243184  0.200124  0.295974   
 1685  0.191489  0.377358  0.305970  0.500000  0.666667  0.500000  0.048973   
 1686  0.191489  0.339623  0.339552  0.529343  0.654001  0.505298  0.283686   
 1687  0.127660  0.698113  0.343284  0.975590  0.666667  1.000000  0.811306   
 
            TUE  
 0     0.000000  
 1    

## 3. Calculate the Euclidean distances between each pair of values

In [6]:
distances_values = dict()

for name in SYNTHESIZERS :
    #distances = distance.cdist(scaled_data[name].values, real_data_scaled, 'euclidean')
    distances_values[name] = pairwise_euclidean_distance(scaled_data[name].values, scaled_data['Real'].values)
    
distances_values

{'GM': '0.9171 ± 0.2255',
 'SDV': '0.9007 ± 0.2035',
 'CTGAN': '0.8999 ± 0.237',
 'WGANGP': '1.1071 ± 0.2559'}

## 4. Calculate the Hausdorff distance between synthetic data and real data

In [7]:
hausdorff_values = dict()

for name in SYNTHESIZERS :
    hausdorff_values[name] = hausdorff_distance(scaled_data[name].values, scaled_data['Real'].values)
    
hausdorff_values

{'GM': 0.7162, 'SDV': 0.6705, 'CTGAN': 0.6604, 'WGANGP': 0.7129}

## 5. Calculate maximum RTS similarity

In [8]:
str_values = dict()

for name in SYNTHESIZERS :
    str_values[name] = rts_similarity(scaled_data[name].values, scaled_data['Real'].values)
    
str_values

{'GM': {'min': 0.0554, 'mean': 0.8038, 'max': 0.9994},
 'SDV': {'min': 0.1096, 'mean': 0.8161, 'max': 0.9977},
 'CTGAN': {'min': 0.0242, 'mean': 0.8023, 'max': 0.9992},
 'WGANGP': {'min': 0.0331, 'mean': 0.7968, 'max': 0.9955}}